# purge haplotigs and overlaps in an assembly based on read depth



In [44]:
#Define Variables
WKDIR=/workspace/hraijc/BB_M7_Assembly
OUTDIR=${WKDIR}/05_Purge_dups
OUTPREFIX=M7_FlyeAll
ASSEMBLY=${WKDIR}/03_FLYE/all/assembly.fasta
ALLREADS=/workspace/hraijc/BB_M7_Assembly/02_M7_cleanReads/M7_all_clean.fastq
PURGEDUPS_BIN_DIR=/workspace/hraijc/git_clones/purge_dups/bin

In [45]:
mkdir -p ${WKDIR}/log/
cd ${OUTDIR}

In [8]:
module load pfr-python3

python /workspace/hraijc/git_clones/purge_dups/scripts/pd_config.py --help

usage: pd_config.py [-h] [-s SRF] [-l LOCD] [-B] [-n FN] [--version]
                    ref pbfofn

generate a configuration file in json format

positional arguments:
  ref                   reference file
  pbfofn                list of pacbio file (one absolute file path per line)

optional arguments:
  -h, --help            show this help message and exit
  -s SRF, --srfofn SRF  list of short reads files (one record per line, the
                        record is a tab splitted line of abosulte file path
                        plus trimmed bases, refer to
                        https://github.com/dfguan/KMC) [NONE]
  -l LOCD, --localdir LOCD
                        local directory to keep the reference and lists of the
                        pacbio, short reads files [.]
  -B, --skipB           skip running busco [False]
  -n FN, --name FN      output config file name [config.json]
  --version             show program's version number and exit


In [16]:
# make pbofn file
printf "/workspace/hraijc/BB_M7_Assembly/02_M7_cleanReads/M7_all_clean.fastq" > ${OUTDIR}/ONT_files.txt

In [36]:
module load pfr-python3
python /workspace/hraijc/git_clones/purge_dups/scripts/pd_config.py -n config.${OUTPREFIX}.json ${ASSEMBLY} ${OUTDIR}/ONT_files.txt

In [38]:

sbatch << EOF
#!/bin/bash
#SBATCH -J PurgeDups
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=24
#SBATCH --mem=20G
#SBATCH --time=31:00:00

module load pfr-python3
module load zlib
module load minimap2
module load BUSCO


python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py config.M7_FlyeAll.json ${PURGEDUPS_BIN_DIR} M7_FlyeAll -p bash

EOF

Submitted batch job 333063


In [48]:
# Peek output
du -sh ${OUTDIR}/M7_FlyeAll/seqs/M7_FlyeAll.purged.fa


850M	/workspace/hraijc/BB_M7_Assembly/05_Purge_dups/M7_FlyeAll/seqs/M7_FlyeAll.purged.fa


In [49]:
# QC results

sbatch << EOF
#!/bin/bash
#SBATCH -J hic_hap_mapyahs
#SBATCH -o ${OUTDIR}/M7_FlyeAll.purged_stats.txt
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem=1G
#SBATCH --time=0:10:00

module load BBMap/38.33

statswrapper.sh in=${OUTDIR}/M7_FlyeAll/seqs/M7_FlyeAll.purged.fa format=3
EOF

Submitted batch job 333195


In [ ]:
module load BUSCO/v5.2.2
bsub -J busco_M7_Q20_Flye_RE3 -o ${WKDIR}/log/busco_M7_Q20_Flye_RE3.log -e ${WKDIR}/log/busco_M7_Q20_Flye_RE3.err -n 25 \
"busco -i /workspace/hraijc/BB_M7_Assembly/03_FLYE/all_q20_re3/assembly.fasta -l eudicots -o Flye04_busco -m geno -c 24 --datasets_version odb10"
module unload BUSCO

In [50]:
# Using the yahs script:
# Contact map after yahs

sbatch << EOF
#!/bin/bash
#SBATCH -J Busco
#SBATCH -o ${OUTDIR}/M7_FlyeAll.purged_stats.txt
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=24
#SBATCH --mem=1G
#SBATCH --time=2:10:00

module load BUSCO/v5.2.2

busco -i ${OUTDIR}/M7_FlyeAll/seqs/M7_FlyeAll.purged.fa -l eudicots -o M7_FlyeAll.purged_busco -m geno -c 24 --datasets_version odb10

EOF

Submitted batch job 333201
